In [2]:
import json
import requests
import datetime

root = 'http://192.168.99.100:9201'
headers = {'Content-Type':'Application/json'}

In [4]:
requests.put(root + '/bigdata-commits',headers=headers, data=json.dumps(
        {
            "mappings": {
            "_default_": {
                "numeric_detection": 'true',
                "date_detection": 'true' ,           
                 "dynamic_templates": [
                  {
                  "lesnames": {
                    "path_match": "*name",
                    "mapping": {
                      "type": "keyword",
                      "analyzer": "standard"
                    }
                  }
                },
                {
                  "lesdates": {
                    "path_match": "*date",
                    "mapping": {
                      "type": "date",
                      "format": "YYYY-MM-DD HH:mm:SS"
                    }
                  }
                },
                {
                  "lereste": { 
                    "match_mapping_type": "string",
                    "mapping": {
                      "type": "text",
                      "analyzer": "standard"
                    }
                  }
                }
              ]
            
            }
          }
        }
)).json()

{'acknowledged': True, 'index': 'bigdata-commits', 'shards_acknowledged': True}

In [6]:
# the difference with the one in server is the date format. It doesn't accept UTC
headers = {'Content-Type':'Application/json'}
mydata =""
madataset = open('commitsAllTable.json', 'r', 1024*1024*16)
i = 1
for line in madataset: 
    jsonLine = json.loads(line)
    jsonLine["author_date"] = str(jsonLine["author_date"]).replace(" UTC", "")
    jsonLine["committer_date"] = str(jsonLine["committer_date"]).replace(" UTC", "")
    fullLine = json.dumps({"index": {"_index": "bigdata-commits", "_type":"data", "_id": i}}) + '\n' + json.dumps(jsonLine) + '\n'
    i = i + 1
    mydata = mydata + fullLine
    if ((i % 10000) == 0):
        requests.post(root + '/_bulk', headers=headers, data=mydata.encode('utf-8'))
        print(str(datetime.datetime.now()) + ", " + str(i))
        mydata = ""
        
requests.post(root + '/_bulk', headers=headers, data=mydata.encode('utf-8'))

2018-03-17 23:32:06.267162, 10000
2018-03-17 23:32:12.481469, 20000
2018-03-17 23:32:18.270727, 30000
2018-03-17 23:32:23.132305, 40000
2018-03-17 23:32:30.405324, 50000
2018-03-17 23:32:36.074361, 60000


<Response [200]>

In [3]:
requests.delete(root + '/bigdata-commits')

<Response [404]>